In [ ]:
# 目前遇到的問題是不會資料預處理 
# 將資料存到numpy 的 npz檔 之後
# 要做image generate

In [ ]:
# 連結google
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#  載入套件
from glob import glob
import cv2
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array
from tqdm.auto import tqdm
import keras
import os
import csv
from shutil import copyfile
from keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten
from keras.models import Model
from sklearn.model_selection import train_test_split
import seaborn as sns
import pandas as pd

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/AIMango/train.csv", encoding="utf-8")
df["label"].value_counts()

B    2068
A    1792
C    1740
Name: label, dtype: int64

In [ ]:
cls = {"A":0, "B":1, "C":2}

In [ ]:
# x_train處理
data = np.empty((5600,224,224,3),dtype=np.float32)
label = np.empty((5600,),dtype=np.float64)
IMG_SIZE = 224

with open("/content/drive/My Drive/Colab Notebooks/AIMango/train.csv", "r", encoding="utf-8") as file:
  csvfile = csv.reader(file)
  next(csvfile)
  
  for i, row in tqdm(enumerate(csvfile)):
    label[i] = cls[row[1]]
    # if row[1] == "A":
    #   label.append(0)
    # elif row[1] == "B":
    #   label.append(1)    
    # elif row[1] == "C":
    #   label.append(2)

    img = cv2.imread('/content/drive/My Drive/Colab Notebooks/AIMango/C1-P1_Train/'+ row[0])
    img = img[:,:,::-1]
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    # img = img/255.0   因為caffe,tf,pythorch用的不一樣所以不要先/255
    img = img_to_array(img)
    data[i] =img

      


In [ ]:
# 處理y_train資料
from tensorflow.keras.utils import to_categorical
# for i in range(len(data)):
#   data[i] = data[i].astype("float32")
y_train = to_categorical(label, num_classes=3)
# y_train = np.array(y_train)

In [ ]:
print(y_train[0])
print(data[0])

[0. 0. 1.]
[[[223. 196.  43.]
  [222. 196.  37.]
  [220. 195.  32.]
  ...
  [  1.   1.   0.]
  [  4.   4.   2.]
  [  2.   2.   0.]]

 [[221. 195.  38.]
  [220. 194.  37.]
  [218. 192.  33.]
  ...
  [  3.   3.   2.]
  [  3.   3.   1.]
  [  1.   1.   0.]]

 [[220. 195.  33.]
  [219. 193.  36.]
  [217. 191.  36.]
  ...
  [  3.   3.   1.]
  [  3.   3.   1.]
  [  3.   3.   1.]]

 ...

 [[220. 181.   0.]
  [225. 187.   2.]
  [228. 190.   2.]
  ...
  [101.  76.  80.]
  [104.  79.  83.]
  [ 99.  74.  78.]]

 [[220. 181.   2.]
  [221. 182.   0.]
  [225. 187.   2.]
  ...
  [ 99.  74.  78.]
  [100.  75.  79.]
  [104.  79.  83.]]

 [[218. 178.   2.]
  [219. 180.   0.]
  [223. 184.   0.]
  ...
  [100.  75.  79.]
  [ 99.  74.  78.]
  [100.  75.  79.]]]


In [ ]:
np.savez('/content/drive/My Drive/Colab Notebooks/AIMango/train1_preprocess.npz', x=data, y=y_train)

In [ ]:
# 確認train資料能讀取
train = np.load('/content/drive/My Drive/Colab Notebooks/AIMango/train1_preprocess.npz')

In [ ]:
x_train = train['x']
y_train = train['y']

In [ ]:
print(x_train[0])
print(y_train[0])

In [ ]:
# x_test處理
data = np.empty((800,224,224,3),dtype=np.float32)
label = np.empty((800,),dtype=np.float64)
IMG_SIZE = 224

with open("/content/drive/My Drive/Colab Notebooks/AIMango/dev.csv", "r", encoding="utf-8") as file:
  csvfile = csv.reader(file)
  next(csvfile)

  for i, row in tqdm(enumerate(csvfile)):
    label[i] = cls[row[1]]
    # if row[1] == "A":
    #   label.append(0)
    # elif row[1] == "B":
    #   label.append(1)    
    # elif row[1] == "C":
    #   label.append(2)
    

    img = cv2.imread('/content/drive/My Drive/Colab Notebooks/AIMango/C1-P1_Dev/'+row[0])
    img_path = row[0]
    img2 = img[:,:,::-1]
    res = cv2.resize(img2, (IMG_SIZE, IMG_SIZE))
    # img = img/255.0 因為caffe,tf,pythorch用的不一樣所以不要先/255
    res = img_to_array(res)
    data[i] = res
  
      

In [ ]:
print(data[0].dtype)

float32


In [ ]:
# 處理y_test資料
from tensorflow.keras.utils import to_categorical
for i in range(len(data)):
  # datas[i] = datas[i].astype("float32")/255
  data[i] = data[i].astype("float32")
y_test = to_categorical(label, num_classes=3)
y_test = np.array(y_test)

In [ ]:
print(data[0].dtype)
print(y_test[0])

float32
[0. 1. 0.]


In [ ]:
np.savez('/content/drive/My Drive/Colab Notebooks/AIMango/test1_preprocess.npz', x=data, y=y_test)

In [ ]:
# 確認test資料能讀取
test = np.load('/content/drive/My Drive/Colab Notebooks/AIMango/test1_preprocess.npz')

In [ ]:
x_test = test['x']
y_test = test['y']

In [ ]:
print(x_test[0].dtype)
print(y_test[0])

float32
[0. 1. 0.]
